# BERTを用いたレビュー文章に対する感情分析モデルの実装と学習・推論

BERTを使用し、IMDbデータのポジ・ネガを分類するモデルを学習させ、推論する。<br>
また推論時のSelf-Attentionを可視化する。

## 学習目標

1.	BERTのボキャブラリーをtorchtextで使用する実装方法を理解する
2.	BERTに分類タスク用のアダプターモジュールを追加し、感情分析を実施するモデルを実装できる
3.	BERTをファインチューニングして、モデルを学習できる
4.  BERTのSelf-Attentionの重みを可視化し、推論の説明を試みることができる

## Library

In [1]:
import random
import time
import numpy as np
from tqdm import tqdm
import torch
from torch import nn
import torch.optim as optim
import torchtext

OSError: dlopen(/Users/eri/opt/anaconda3/lib/python3.7/site-packages/torchtext/_torchtext.so, 6): Library not loaded: @rpath/libtorch_cpu.dylib
  Referenced from: /Users/eri/opt/anaconda3/lib/python3.7/site-packages/torchtext/_torchtext.so
  Reason: image not found

In [ ]:
# 乱数のシードを設定
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

## IMDbデータを読み込み、DataLoaderを作成（BERTのTokenizerを使用）

In [1]:
# 前処理と単語分割をまとめた関数

import re
import string
from utils.bert import BertTokenizer

def preprocessing_text(text):
    '''
    IMDbの前処理
    '''
    # 改行コードを消去
    text = re.sub('<br />', '', text)
    
    # カンマ、ピリオド以外の記号をスペースに置換
    for p in string.punctuation:
        if (p == '.') or (p == ','):
            continue
        else:
            text = text.replace(p, ' ')
            
    # ピリオドとカンマの前後にスペースを入れておく
    text = text.replace('.', ' . ')
    text = text.replace(',', ' , ')
    return text

# 単語分割用のTokenizerを用意
# 大文字は小文字に変換する
tokenizer_bert = BertTokenizer(vocab_file='./vocab/bert-base-uncased-vocab.txt', do_lower_case=True)

# 前処理と単語分割をまとめた関数
# 単語分割の関数を渡すので、tokenizer_bertではなくtokenizer_bert.tokenizeを渡す！
def tokenizer_with_preprocessing(text, tokenizer=tokenizer_bert.tokenize):
    text = preprocessing_text(text)
    ret = tokenizer(text)
    return ret

In [ ]:
# データを読み込んだときに行う処理
max_length = 256
TEXT = torchtext.data.Field(sequential=True, tokenize=tokenizer_with_preprocessing, use_vocab=True,
                           lower=True, include_lengths=True, batch_first=True, fix_lnegth=max_length)
LABEL = torchtext.data.Field(sequential=False, use_vocab=False)


In [ ]:
# 各tsvファイルの読み込み
train_val_ds, test_ds = torchtext.data.TabularDataset.splits(
    path='./data/',
    train='IMDb_train.tsv',
    test='IMDb_test.tsv',
    format='tsv',
    fields=[('Text', TEXT), ('Label', LABEL)])

# torchtext.data.Datasetのsplit関数で訓練データとvalidationデータを分ける
train_ds, val_ds = train_val_ds.split(split_ratio=0.8, random_state=random.seed(1234))


In [ ]:
# bertのvocabraryは全単語を使用する
# 訓練データからvocabraryは作成しない

# bert用の単語辞書を辞書型変数に変換する
from utils.bert import BertTokenizer, load_vocab

vocab_bert, ids_to_tokens_bert = load_vocab(vocab_file='./vocab/bert-base-uncased-vocab.txt')

# 一度bulild_vocabを実行しないとTEXTオブジェクトがvocabのメンバ変数をもってくれない,　エラーを吐く
# 一度適当にbuild_vocabでボキャブラリーを作成してからBERTのボキャブラリーを上書きする
TEXT.build_vocab(train_ds, min_freq=1)
TEXT.vocab.stoi = vocab_bert


In [ ]:
# DataLoaderを作成する
batch_size = 32  # bertでは16, 32あたりを使う

train_dl = torchtext.data.Iterator(train_ds, batch_size=batch_size, train=True)
val_dl = torchtext.data.Iterator(val_ds, batch_size=batch_size, train=False, sort=False)
test_dl = torchtext.data.Iterator(test_ds, batch_size=batch_size, train=False, sort=False)


# 辞書オブジェクトにまとめる
dataloaders_dict = {'train': train_dl, 'val': val_dl}



In [ ]:
# 動作確認　検証データのデータセットで確認
batch = next(iter(val_dl))
print(batch.Text)
print(batch.Label)

In [ ]:
# ミニバッチの１文目を確認してみる
text_minibatch_1 = (batch.Text[0][1]).numpy()  # なぜ[1]

# IDを単語に戻す
text = tokenize_bert.convert-_ids_to_tokens(text_minibatch_1)

print(text)
# サブワードで分割されていることがわかる

## 感情分析用のBERTモデルを構築

In [ ]:
from utils.bert import get_config, BertModel, set_learned_params

# モデルの設定のJSONファイルをオブジェクトとして読み込み
config = get_config(file_path='./weights/pytorch_config.bin')
# BERTの基本モデルを作成
net_bert = BertModel(config)

# BERTモデルに学習済みパラメータをセットする
net_bert = set_learned_params(net_bert, weights_path='./weights/pytorch_model.bin')

In [2]:
class BertForIMDb(nn.Module):
    '''
    BERTモデルにIMDｂのネガ・ポジを判定する部分(全結合層)をつなげたモデル
    '''
    
    def __init__(self, net_bert):
        super(BertForIMDb, self).__init__()
        
        # bertモジュール
        self.bert = net_bert
        self.cls = nn.Linear(in_features=768, out_features=2)
        
        # 重み初期化
        # モジュールによって場合分けが必要なければこんなふうにシンプルに書ける
        nn.init.normal_(self.cls.weight, std=0.02)
        nn.init.normal_(self.cls.bias, 0)
        
    def forward(self, input_ids, token_type_ids=None, attention_mask=None, output_all_encoded_layers=False, attention_show_flg=False):
        '''
        input_ids： [batch_size, sequence_length]の文章の単語IDの羅列
        token_type_ids： [batch_size, sequence_length]の、各単語が1文目なのか、2文目なのかを示すid
        attention_mask：Transformerのマスクと同じ働き
        output_all_encoded_layers：最終出力に12段のTransformerの全部をリストで返すか、最後だけかを指定
        attention_show_flg：Self-Attentionの重みを返すかのフラグ
        '''
        
        # bertの基本モデル部分の順伝播
        # 今回pooled_outputは使用しない
        if attention_show_flg == True:
            encoded_layers, pooled_output, attention_probs = self.bert(
                input_ids, token_type_ids, attention_mask, output_all_encoded_layers, attention_show_flg)
        elif attention_show_flg == False:
            encoded_layers, pooled_output = self.bert(
                input_ids, token_type_ids, attention_mask, output_all_encoded_layers, attention_show_flg)
            
        # 入力文章の１単語目[CLS]　の特徴量を使用してポジ・ネガを分類する
        vec_0 = encoded_layers[:,0,:]
        vec_0 = vec_0.view(-1, 768)  # [batch_size, hidden_size]
        out = self.cls(vec_0)
        
        # attention_showのときはattention_probsもリターン
        if attention_show_flg == True:
            return out, attention_probs
        elif attention_show_flg == False:
            return out

NameError: name 'nn' is not defined

In [ ]:
# モデル構築
net = BertForIMDb(net_bert)
net.train()
print('ネットワーク設定完了')

## BERTのファインチューニングに向けた設定

In [ ]:
# 勾配計算を最後のBertLayerモジュールと追加した分類アダプターのみ実行

# 全てのモジュールを勾配計算Falseにする
for name, param in net.named_parameters():
    param.requires_grad = False
    
# 最後のBertLayerモジュールの勾配計算をTrueにする
for name, param in net.bert.encoder.layer[-1].named_parameters():
    param.requires_grad = True
    
# 識別器を勾配計算Trueに変更
for name, param in net.cls.named_parameters():
    param.requires_grad = True

In [ ]:
# 最適化手法の設定
# bertの元の部分(識別機でない部分)はファインチューニング
optimizer = optim.Adam([
    {'params': net.bert.encoder.layer[-1].parameters(), 'lr': 5e-5},
    {'params': net.cls.parameters(), 'lr': 5e-5}
], betas=(0.9, 0.999))

# 損失関数の設定
criterion = nn.CrossEntropyLoss()

## 学習・検証の実施

In [3]:
def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):
    # GPUが使えるか確認
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    print('使用デバイス：', device)
    print('-----start-----')
    
    net.to(device)
    
    torch.backends.cudnn.benchmark = True
    
    batch_size = dataloaders_dict['train'].batch_size
    
    # epochのループ
    for epoch in range(num_epochs):
        # epochごとの訓練と検証のループ, モデルのモードを変更する
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()
            else:
                net.eval()
                
            epoch_loss = 0.0
            epoch_corrects = 0  # 分類なので
            iteration = 1

            t_epoch_start = time.time()
            t_iter_start = time.time()

            # ミニバッチを取り出すループ
            for batch in (dataloaders_dict[phase]):
                # batchはTextとLabelの辞書オブジェクト

                # データをデバイスに送る
                inputs = batch.Text[0].to(device)
                labels = batch.Label.to(device)

                # optimizerの初期化  これはvalのときは必要なさそう
                optimizer.zero_grad()

                # forward
                with torch.set_grad_enabled(phase=='train'):
                    # BertForIMDbに入力
                    outputs = net(inputs,  token_type_ids=None, attention_mask=None, 
                                 output_all_encoded_layers=False, attention_show_flg=False)

                    loss = criterion(oututs, labels)
                    _, preds = torch.max(outputs, 1)

                    # 訓練時はbackpropagation
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                        if (iteration%10 == 0):
                            t_iter_finish = time.time()
                            duration = t_iter_finish - t_iter_start
                            acc = (torch.sum(preds==labels.data)).double()/batch_size
                            print('イテレーション　{} || Loss: {:.4f} || 10iter: {:.4f} sec. || 本イテレーションの正解率: {}'.format(iteration, loss.item(), duration, acc))
                            t_iter_start = time.time()

                    iteration += 1
                
                    # 損失と正解数の合計を計算
                    # lossは平均で返ってきているからbatch_sizeをかける
                    # もともとのbatch＿sizeじゃなくてDataloaderから取ってきたbatch_sizeのほうがいいのでは？
                    epoch_loss += loss.item()*batch_size  
                    epoch_corrects += torch.sum(preds=labels.data)

            # epochごとのlossと正解率
            t_epoch_finish = time.time()
            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
            epoch_acc = epoch_corrects.double() / len(dataloaders_dict[phase].dataset)
            
            print('Epoch{} / {} | {:.5f} | Loss: {:.4f} Acc: {:.4f}'.format(epoch+1, num_epochs, phase, epoch_loss, epoch_acc))
            t_epoch_start = time.time()
            
    return net

In [ ]:
# 学習・検証
# 2epochで４０分ほど
num_epochs = 2
net_trained = train_model(net, dataloaders_dict, criterion, optimizer, num_epochs=num_epochs)


In [ ]:
# 学習したパラメータを保存
save_path = './weights/bert_fine/tuning_IMDb.pth'
torch.save(net_trained.state_dict(), save_path)

In [ ]:
# テストデータでの正解率を求める
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

net_trained.eval()  # モデルを検証モードに
net_trained.to(device)

# epochの正解数を記録する変数
epoch_corrects = 0

for batch in tqdm(test_dl):  # testデータのDataLoader
    # batchはTextとLabelの辞書オブジェクト

    inputs = batch.Text[0].to(device)
    labels = batch.Label.to(device)
    
    with torch.set_grad_enable(False):
        # BertForIMDbに入力
        outputs = net_trained(inputs, token_type_ids=None, attention_mask=None, output_all_encoded_layers=False, attention_show_flg=False)
        loss = criterion(outputs, labels)
        _, preds = torch.max(outputs, 1)
        epoch_corrects += torch.sum(preds==lables.data)  # すべてのデータに対して正解した合計を求めることになる
        
# 正解率
epoch_acc = epoch_corrects.double() / len(test_dl.dataset)
print('テストデータ{}個での正解率：{:.4f}'.format(len(test_dl.dataset), epoch_acc))

## Attentionの可視化

In [ ]:
# batch_sizeを６４にしたテストデータでDataLoaderを作成
batch_size = 64
test_dl = torchtext.data.Iterator(test_ds, batch_size=batch_size, train=False, sort=False)

In [ ]:
batch = next(iter(test_dl))

inputs = batch.Text[0].to(device)
labels = batch.Label.to(device)

outputs, attention_probs = net_trained(inputs, token_type_ids=None, attention_mask=None,
                                      output_all_encoded_layers=False, attention_show_flg=True)

_, preds = torch.max(outputs, 1)

In [4]:
# HTMLを作成する関数を実装
def highlight(word, attn):
    '''
    Attentionの値が大きいと文字の背景が濃い赤になるhtmlを出力させる関数
    '''
    
    html_color = '#%02X%02X%02X' % (255, int(255*(1 - attn)), int(255*(1 - attn)))
    return '<span style="background-color: {}"> {}</span>'.format(html_color, word)

def mk_html(index, batch, preds, normalized_weights, TEXT):
    '''
    HTMLデータを作成する
    '''
    
    # indexの結果を抽出
    sentence = batch.Text[0][index]
    label = batch.Label[index]
    pred = preds[index]
    
    # ラベルと予測結果を文字に置き換え
    if label == 0:
        label_str = 'Negative'
    else:
        label_str = 'Positive'
        
    if pred == 0:
        pred_str = 'Negative'
    else:
        pred_str = 'Positive'
        
    # 表示用のhtmlを作成
    html = '正解ラベル:{}<br>推論ラベル: {}<br><br>'.format(label_str, pred_str)
    
    # self-attentionの重みを可視化. multi-headが１２個なので１２種類のアテンションが存在する
    for i in range(12):
        # indexのAttentionを抽出と規格化
        # ０単語目[CLS]のi番目のmulti-head attentionを取り出す
        # indexはミニバッチの何番目のデータ化を示す
        attens = normalized_weights[index, i, 0, :]
        attens /= attens.max()
        
        html += '[BERTのAttentionを可視化_' + str(i+1) + ']<br>'
        for word, attn in zip(sentence, attens):
            
            # 単語が[SEP]の場合は文章が終わりなのでbreak
            if tokenizer_bert.convert_ids_to_tokens([word,numpy().tolist()])[0] == '[SEP]':
                break
            
            # 関数hightlightで色を付ける
            # 関数tokenizer_bert.convert_ids_to_tokensでIDを単語に戻す
            html += highlight(tokenizer_bert.convert_ids_to_tokens([word.numpy().tolist()])[0], attn)
        html += '<br><br>'
        
    # 12種類のAttentionの平均を求める。最大値で規格化
    all_attens = attens*0
    for i in range(12):
        attens += normalized_weights[index, i, 0, :]
    attens /= attens.max()
    
    html += '[BERTのAttentionを可視化_ALL]<br>'
    for word, attn in zip(sentence, attens):
        # 単語が[SEP]だと文章終わりなのでbreak
        if tokenizer_bert.convert_ids_to_tokens([word.numpy().tolist()])[0] == '[SEP]':
            break
            
        # 関数hightlightで色を付ける
        # 関数tokenizer_bert.convert_ids_to_tokensでIDを単語に戻す
        html += highlight(tokenizer_bert.convert_ids_to_tokens([word.numpy().tolist()])[0], attn)
    html += '<br><br>'
        
    return html
        

In [ ]:
from IPython.display import HTML

index = 3
html_output = mk_html(index, batch, preds, attention_probs, TEXT)  # HTML作成
HTML(html_output)

In [ ]:
# Transformerではうまくいかなかった文章の推論結果とAttention
index = 61
html_output = mk_html(index, batch, preds, attention-probs, TEXT)
HTML(html_output)